In [ ]:
#!/usr/bin/env python
#############################################
# This creates figure 
# SST2-prec-vs-wind.png
# in paper
############################################

import sys
# import modules in other directories
sys.path.append('../Utils/')


import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


import trax_util as trx
import PlotUtil as Pu
import humiditycalcs as Hum

import importlib

import time as TimeUtils

import sys
# import modules in other directories
sys.path.append('/glade/work/juliob/PyRegridding/Regridder/')
import argparse as arg


import esmfRegrid as erg


importlib.reload( trx )
importlib.reload( Pu )


In [ ]:
##################
# What field do you
# want look at jointly
# with Storm Precip
###################
YfldA='wind'

In [ ]:
##################
# Get track file
###################
sstA='sst2'
power_wind=(1./6.)**(0.11)
TrakFile1 = trx.rcp85fname(sst=sstA) 
BaseName1 = trx.rcp85fname(sst=sstA, justBaseName=True) 
print(TrakFile1)
print(BaseName1)
trk1=trx.readtrx( TrakFile1  , power_wind=power_wind )


ens=1
power_wind=(1./6.)**(0.11)
TrakFile0 = trx.pdfname(ens=ens) 
BaseName0 = trx.pdfname(ens=ens, justBaseName=True) 
trk0=trx.readtrx( TrakFile0  , power_wind=power_wind )


In [ ]:
###############################
# Add in storm precip on tracks
###############################
npF1='/glade/work/juliob/NumPySaves/PrecTrax/'+BaseName1+'-prectrax-2070-2099.npz'
try:
    trk1_addF=np.load(npF1)
except FileNotFoundError:
    print(f'The file {npF1} was not found.')
    trk1_addF = None  # Or some other action

npF0='/glade/work/juliob/NumPySaves/PrecTrax/'+BaseName0+'-prectrax-1979-2012.npz'
try:
    trk0_addF=np.load(npF0)
except FileNotFoundError:
    print(f'The file {npF0} was not found.')
    trk0_addF = None  # Or some other action


if  (trk0_addF != None):
    trk0['prectrk']=1000.*86400.*trk0_addF['prectrk']
if  (trk1_addF != None):
    trk1['prectrk']=1000.*86400.*trk1_addF['prectrk']

    
print(trk0.keys())
print(trk1.keys())


In [ ]:
###############################
# Add in storm T850 on tracks
###############################
npF1='/glade/work/juliob/NumPySaves/T850Trax/'+BaseName1+'-t850trax-2070-2099.npz'
try:
    trk1_t850=np.load(npF1)
except FileNotFoundError:
    print(f'The file {npF1} was not found.')
    trk1_t850 = None  # Or some other action

npF0='/glade/work/juliob/NumPySaves/T850Trax/'+BaseName0+'-t850trax-1979-2012.npz'
try:
    trk0_t850=np.load(npF0)
except FileNotFoundError:
    print(f'The file {npF0} was not found.')
    trk0_t850 = None  # Or some other action


if  (trk0_t850 != None):
    trk0['t850trk']=trk0_t850['t850trk']
if  (trk1_t850 != None):
    trk1['t850trk']=trk1_t850['t850trk']

    
print(trk0.keys())
print(trk1.keys())


In [ ]:
if (sstA == 'sst4'):
    trk1=trx.Heal_rc4( trk1 )
    print(f" 'Fixed' years in SST4 tracks. Might wanna check " )
if (sstA == 'sst6'):
    trk1=trx.Heal_rc6( trk1 )
    print(f" 'Fixed' years in SST6 tracks. Might wanna check " )


In [ ]:
oo=np.where( trk1.year.flatten() > 0)
yrmin=np.nanmin( trk1.year.flatten()[oo[0]] )
yrmax=np.nanmax( trk1.year.flatten()[oo[0]] )
Nyrs = yrmax-yrmin+1
print( yrmin,yrmax )

In [ ]:
##
oo=np.where( trk0.year.flatten() > 0)
#plt.plot( trk0.prectrk.flatten()[oo[0]] )
T850_PDmean=trk0.t850trk.flatten()[oo[0]].mean()
print( T850_PDmean )
Ht850,tee=np.histogram( trk0.t850trk.flatten()[oo[0]] , bins=1001, range=(280,300))
plt.plot( tee[1:],Ht850 )

In [ ]:
#######################
# Set range for 2D PDFs
########################
if (YfldA=='wind'):
    pdf_range = [(2,160.),(0.,80.)] 
    titlfrag = 'Storm Precip vs Wind'
    yaxlab = 'Wind (ms$^{-1}$)'
if (YfldA=='pres'):
    pdf_range = [(2,160.),(850.,1030.)] 
    titlfrag = 'Storm Precip vs Central Pressure'
    yaxlab = 'Central Pressure (hPa)'


#######################
# Make first PDFs (unscaled precip)
#######################
B=-1
oo = np.where( trk0.basin.flatten()>B )
x,y= trk0.prectrk.flatten()[oo[0]], trk0[YfldA].flatten()[oo[0]]
H0,x0,y0=np.histogram2d(x, y, bins=40   ,range=pdf_range )

H0  =  H0/np.sum(H0)

oo = np.where( trk1.basin.flatten()>B )
x,y= trk1.prectrk.flatten()[oo[0]], trk1[YfldA].flatten()[oo[0]]
H1,x1,y1=np.histogram2d(x, y, bins=40   ,range=pdf_range )

H1  =  H1/np.sum(H1)

In [ ]:
qsatscale1 = Hum.qsat(85_000., trk1.t850trk) / Hum.qsat(85_000., 290. )
qsatscale0 = Hum.qsat(85_000., trk0.t850trk) / Hum.qsat(85_000., 290. )


In [ ]:
B=-1
oo = np.where( trk0.basin.flatten()>B )
x,y= (trk0.prectrk / qsatscale0 ).flatten()[oo[0]], trk0[YfldA].flatten()[oo[0]]
Hs0,x0,y0=np.histogram2d(x, y, bins=40   ,range=pdf_range )

Hs0  =  Hs0/np.sum(Hs0)

oo = np.where( trk1.basin.flatten()>B )
x,y= (trk1.prectrk / qsatscale1).flatten()[oo[0]], trk1[YfldA].flatten()[oo[0]]
Hs1,x1,y1=np.histogram2d(x, y, bins=40   ,range=pdf_range )

Hs1  =  Hs1/np.sum(Hs1)

In [ ]:
import matplotlib.patches as mpatches

levels=np.array([1,2,5,10,20,50,100,200,500,1000])*1.e-5
widths=np.array([1,3,1,3,1 ,3 ,1  ,3.  ,1,  3])

lablev=levels[1::8]
#print(lablev)
cmap=Pu.step_cmap(levels=levels,cmapN='gist_ncar')
#plt.contourf(H1,levels=levels,cmap=cmap)
fig=plt.figure( figsize=(20,5))
ax=fig.add_subplot(1,2,2)
co=ax.contour(y1[1:],x1[1:],Hs1,levels=levels, colors='red' ,linewidths=widths) #,cmap=cmap,alpha=.5)
cB=ax.contour(y0[1:],x0[1:],Hs0,levels=levels, colors='black',linewidths=widths) #,cmap=cmap,alpha=.5)
cl=ax.contour(y0[1:],x0[1:],Hs0,levels=levels, colors='blue',linewidths=widths) #,cmap=cmap,alpha=.5)

specific_levels = [0.01,0.0005  ] #,0.0001]
labels = co.clabel(inline=True, fontsize=10, levels=specific_levels)
# Add a background box around each label
for label in labels:
    label.set_bbox(dict(facecolor='white', edgecolor='none', pad=2))

labels = cl.clabel(inline=True, fontsize=10, levels=specific_levels  ) #, manual=[(5,18),(20,10)])
# Add a background box around each label
for label in labels:
    label.set_bbox(dict(facecolor='white', edgecolor='none', pad=2))


cbar=plt.colorbar(cB,ax=ax,shrink=0.8)
cbar.set_label( 'Probability',size=14)
ax.set_ylabel( "500km Precip (mm d$^{-1}$)" ,size=14)
ax.set_xlabel( yaxlab ,size=14)
ax.set_title("PD and "+sstA.upper() + ": " + titlfrag + ", Scaled by $q_{sat}(T_{850})$",size=16 )

# Now we add labels to the contours at these levels
#plt.clabel(cl, levels=lablev, inline=True, fontsize=12)
ax.text(-0.12, 1.1, f"{chr(97 +1)})", transform=ax.transAxes,
    fontsize=20, fontweight='bold', va='top')

proxy1 = mpatches.Patch(color='blue', label='PD')
proxy2 = mpatches.Patch(color='red', label=sstA.upper())
# Create a legend
plt.legend(handles=[proxy1, proxy2])



ax=fig.add_subplot(1,2,1)
co=ax.contour(y1[1:],x1[1:],H1,levels=levels, colors='red' ,linewidths=widths) #,cmap=cmap,alpha=.5)
cB=ax.contour(y0[1:],x0[1:],H0,levels=levels, colors='black',linewidths=widths) #,cmap=cmap,alpha=.5)
cl=ax.contour(y0[1:],x0[1:],H0,levels=levels, colors='blue',linewidths=widths) #,cmap=cmap,alpha=.5)

specific_levels = [0.01,0.0005  ] #,0.0001]
labels = co.clabel(inline=True, fontsize=10, levels=specific_levels)
# Add a background box around each label
for label in labels:
    label.set_bbox(dict(facecolor='white', edgecolor='none', pad=2))

labels = cl.clabel(inline=True, fontsize=10, levels=specific_levels  ) #, manual=[(5,18),(20,10)])
# Add a background box around each label
for label in labels:
    label.set_bbox(dict(facecolor='white', edgecolor='none', pad=2))

cbar=plt.colorbar(cB,ax=ax,shrink=0.8)
cbar.set_label( 'Probability',size=14)
ax.set_ylabel( "500km Precip (mm d$^{-1}$)" ,size=14)
ax.set_xlabel( yaxlab ,size=14)
ax.set_title("PD and "+sstA.upper() +": " + titlfrag + ", Unscaled" ,size=16)
ax.text(-0.12, 1.1, f"{chr(97 +0)})", transform=ax.transAxes,
    fontsize=20, fontweight='bold', va='top')

proxy1 = mpatches.Patch(color='blue', label='PD')
proxy2 = mpatches.Patch(color='red', label=sstA.upper())
# Create a legend
plt.legend(handles=[proxy1, proxy2])


Pu.save_link(png_filename = "SST2-prec-vs-wind.png")


In [ ]:
plt.semilogy( x0[1:],np.average(H0.T , axis=0 ),color='blue' )
plt.semilogy( x1[1:],np.average(H1.T , axis=0 ),color='red' )
plt.xlim(10,160)

# 